In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from pandas import DataFrame

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")


Printing general info about dataframe

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

Dropping useless column.

In [ ]:
df.drop(['customerID'],axis=1,inplace=True)


Just printing unique values of non-binary and non-numeric columns

In [ ]:
df.MultipleLines.unique()

In [ ]:
df.InternetService.unique()

In [ ]:
df.OnlineSecurity.unique()

In [ ]:
df.OnlineBackup.unique()

In [ ]:
df.DeviceProtection.unique()

In [ ]:
df.TechSupport.unique()

In [ ]:
df.StreamingTV.unique()

In [ ]:
df.StreamingMovies.unique()

In [ ]:
df.Contract.unique()

In [ ]:
df.PaymentMethod.unique()

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [ ]:
df.isnull().sum()


Checking, how rows containing some nan values look like.

In [ ]:
is_NaN = df.isnull()
mask = is_NaN.any(axis=1)
nan_rows = df[mask]
print(nan_rows)

And dropping them as well, since there is relatively small number of them.

In [ ]:
df.dropna(inplace=True)



Coding binary features into numeric values and creating one-hot encoding for non-binary non-numeric ones.

In [ ]:
df['Churn'] = df['Churn'].replace('Yes',1)
df['Churn'] = df['Churn'].replace('No',0)

df['gender'] = df['gender'].replace('Male',1)
df['gender'] = df['gender'].replace('Female',0)

df['Partner'] = df['Partner'].replace('No',0)
df['Partner'] = df['Partner'].replace('Yes',1)

df['PhoneService'] = df['PhoneService'].replace('No',0)
df['PhoneService'] = df['PhoneService'].replace('Yes',1)

df['Dependents'] = df['Dependents'].replace('No',0)
df['Dependents'] = df['Dependents'].replace('Yes',1)

df['PaperlessBilling'] = df['PaperlessBilling'].replace('No',0)
df['PaperlessBilling'] = df['PaperlessBilling'].replace('Yes',1)


df_dummies = pd.get_dummies(
    columns = [
        'MultipleLines',
        'InternetService',
        'OnlineSecurity',
        'OnlineBackup',
        'DeviceProtection',
        'TechSupport',
        'StreamingTV',
        'StreamingMovies',
        'Contract',
        'PaymentMethod'
    ],
    data = df)


In [ ]:
df_dummies.head()



In [ ]:
sns.countplot(x="Churn",data=df)


In [ ]:
def correlation_matrix_viz(corr, name):

    corr.style.background_gradient(cmap='coolwarm')
    # print(corr)

    # plt.matshow(corr)
    # plt.show()

    f = plt.figure(
        figsize=(15, 15)
    )

    plt.matshow(
        corr,
        fignum=f.number
    )

    plt.xticks(range(corr.shape[1]), corr.columns, fontsize=14, rotation=90)
    plt.yticks(range(corr.shape[1]), corr.columns, fontsize=14)
    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Correlation Matrix " + name, fontsize=16)
    plt.plot()


Just overall look on data correlations. Not providing much information since it is quite big.

In [ ]:

corr_data = df_dummies.corr()
correlation_matrix_viz(corr_data, 'raw')

Correlations of Tenure data feature with others. I am interested into this because higher tenure values reflects a satisfied customer.

In [ ]:
print(corr_data["tenure"])

Potentially bad methodology, since I am looking at all data. This might be done with seen-dataset only. But since I am not optimizing any parameters here, I am leaving it here for now

In [ ]:
churn_correl = corr_data['Churn']
churn_correl = churn_correl.drop(['Churn'])
plt.figure(figsize=(16,16))
churn_correl = churn_correl.sort_values()
churn_correl.plot.bar()
plt.show()

Plotting correlations with predicted value.


Potentially bad methodology, since I am looking at all data. This might be done with seen-dataset only. But since I am not optimizing any parameters here, I am leaving it here for now.



In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, roc_auc_score, f1_score

def evaluation(Y_true, Y_pred, phase_name):
    print(f"Evaluation of data {phase_name}")

    print("Confusion matrix:")
    conf_matrix_test = confusion_matrix(Y_true, Y_pred)
    print(conf_matrix_test)

    print("F1 score:")
    f_1_test = f1_score(Y_true, Y_pred)
    print(f_1_test)

    print("Accuracy:")
    accuracy_test = accuracy_score(Y_true, Y_pred)
    print(accuracy_test)

    print("Precision:")
    precision_test = precision_score(Y_true, Y_pred)
    print(precision_test)

    print("Roc_auc:")
    roc_auc_test = roc_auc_score(Y_true, Y_pred)
    print(roc_auc_test)


def model_evaluation(model_name, Y_train_true, Y_train_pred, Y_test_true, Y_test_pred):
    print("--------------------------------------------------------------------------------")
    print("--------------------------------------------------------------------------------")
    print(f"Evaluation of model: {model_name}")
    evaluation(Y_train_true, Y_train_pred, "train_phase")
    evaluation(Y_test_true, Y_test_pred, "test_phase")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler

Y = df_dummies['Churn']

df_final = df_dummies.drop('Churn', axis=1)

In [ ]:
X = df_final

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)

If I would like to be super methodologically precise, K-fold would be nice here.

In [ ]:
scaler = StandardScaler()
transformed_train = scaler.fit_transform(X_train)
transformed_test = scaler.transform(X_test)

model_01 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000, n_jobs=10, penalty='l2',
                   random_state=None, tol=0.0001, verbose=0,
                   warm_start=False)

model_01.fit(X = transformed_train, y = Y_train)

Y_train_predicted = model_01.predict(transformed_train)
Y_test_predicted = model_01.predict(transformed_test)

model_evaluation("LogisticRegression", Y_train, Y_train_predicted, Y_test, Y_test_predicted)

Logistics regression is my first choice. The model is easy, robust, can't be overfitted and often provides good baseline.

In [ ]:
transposed = model_01.coef_.flatten()

print(transposed.shape)

In [ ]:
pd.DataFrame(
    {
        'Input' : X.columns,
        'Weights to normalized inputs' : transposed
    }).sort_values('Weights to normalized inputs', ascending=False)


In [ ]:
import xgboost as xgb
model_02 = xgb.XGBClassifier(
    n_estimators=2000,
    max_depth=5,
    num_boost_round = 100,
    # gamma =,
    # reg_alpha= 0,
    subsample= 0.8
)

X_train_t, X_valid, Y_train_t, Y_valid = train_test_split(X_train, Y_train, test_size = 0.1)
model_02.fit(X_train_t, Y_train_t, eval_set=[(X_valid, Y_valid)], early_stopping_rounds = 5)

Y_train_predicted = model_02.predict(X_train)
Y_test_predicted = model_02.predict(X_test)

model_evaluation("RandomForest", Y_train, Y_train_predicted, Y_test, Y_test_predicted)

XGBoost returns slightly worse results than logistic regression. K-fold would give us greater confidence which model is better. XBBoost obviously uses validation set to detect overfitting. I've also tried naive bayes model, but it gave me poor results. It might be indicator that Chur value is based on multiple feature interaction.

In [ ]:
pd.DataFrame(
    {
        'Input' : X.columns,
        'Importance' : model_02.feature_importances_
    }).sort_values('Importance', ascending=False)


Importance of features based on XGBoost differs from run to run. K-fold with multiple fits per fold would give us better statistics over what features are important.

I must admit I am using SHAP analysis for the first time. I wanted to try it:

In [ ]:
import shap
explainer = shap.TreeExplainer(model_02)
shap_values = explainer.shap_values(X_train)



np.abs(shap_values.sum(1) + explainer.expected_value - Y_train_predicted).max()
shap.summary_plot(shap_values, X_train)

From correlation matrix, linear regression coefficients and XGBoost analysis we can see that tenure and total charges are highly correlated.
We can also see a positive correlation between customers leaving and monthly payments - which is not surprising nor interesting.

The longer customer uses the product, the lower probability one leaves the Company.

Some naively-bayesian observations after model explanation. We can see probabilities of customer leaving/staying based on three picked data features. (I am using means here since selected data features are in boolean values)

In [ ]:
df_dummies.loc[df_dummies["Churn"] == 0, 'Contract_Month-to-month'].mean()

In [ ]:
df_dummies.loc[df_dummies["Churn"] == 1, 'Contract_Month-to-month'].mean()


In [ ]:
df_dummies.loc[df_dummies["Churn"] == 0, 'InternetService_Fiber optic'].mean()

In [ ]:
df_dummies.loc[df_dummies["Churn"] == 1, 'InternetService_Fiber optic'].mean()


In [ ]:
df_dummies.loc[df_dummies["Churn"] == 0, 'OnlineSecurity_No'].mean()

In [ ]:
df_dummies.loc[df_dummies["Churn"] == 1, 'OnlineSecurity_No'].mean()

We can clearly see that 88+% of leaving customers have month-to-month contract.

A significant majority (69%) of leaving customers also have Fiber optic cable and no online security (78%).

I would say that the Company should try to persuade clients to sign to a longer contracts or give to the customers something extra for start and once customers get used to a product, switch to standard product package.

Also, there is some problem with the optic fiber connection. There might be stronger competition on the market or fiber optic connections are not stable/fast enough for some reason.

I don't know what "Online security" means, but I am guessing it is some sort of firewall. People without online security tend to leave company as well. The Company should probably make "online security" feature as a part of a standard product.


